### Prueba usando one hot encoding con tipos de propiedades y unos hiperparametros sacados de XGBoost_v1.0

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

import re

from math import sqrt

In [3]:
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

In [4]:
df = pd.read_csv("train.csv")

In [7]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [9]:
df.fecha = pd.to_datetime(df.fecha)

In [10]:
df['year'] = df.fecha.dt.year
df['mes'] = df.fecha.dt.month
df['dia_del_anio'] = df.fecha.dt.day

In [15]:
columnsDrop = ['id','titulo','descripcion','direccion','ciudad','provincia','idzona','lat','lng','fecha']

In [16]:
X = df.drop(['precio'] + columnsDrop,axis = 1)
y = df['precio']

In [20]:
X.shape

(240000, 15)

In [22]:
X = pd.get_dummies(X, dummy_na=False)

In [23]:
X.shape

(240000, 38)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [31]:
parametros = {'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 1,
 'n_estimators': 100,
 'nthread': 4,
 'random_state': 0,
 'subsample': 1}

In [28]:
model = XGBRegressor(**parametros)

In [32]:
%%time
results = model.fit(X_train,y_train)

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[19:12:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 52 s, sys: 126 ms, total: 52.1 s
Wall time: 13 s


In [33]:
pred = model.predict(X_test)

In [34]:
RMSLE(y_test, pred)

0.44483587521484125

In [42]:
X_Null = df.drop(['precio'] + columnsDrop,axis = 1)

In [43]:
X_Null = pd.get_dummies(X_Null, dummy_na=True)

In [44]:
X_Null.shape

(240000, 39)

In [45]:
X_Null_train, X_Null_test, yNull_train, yNull_test = train_test_split(X_Null, y, test_size=0.25, random_state=1)

In [46]:
modelNull = XGBRegressor(**parametros)

In [47]:
%%time
resultsWithNulls = modelNull.fit(X_Null_train,yNull_train)

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[19:20:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 50.1 s, sys: 1.64 s, total: 51.7 s
Wall time: 12.9 s


In [48]:
predNull = resultsWithNulls.predict(X_Null_test)

In [49]:
RMSLE(y_test,predNull) #Agregando la columna que considere si es null, la ganancia es imperceptiblet

0.4446278365341396

In [50]:
RMSLE(y_test, pred)

0.44483587521484125